In [1]:
# from llama_index.core import SimpleDirectoryReader

# reader = SimpleDirectoryReader(input_dir="data")

# all_docs = []
# for docs in reader.iter_data():
#     # <do something with the documents per file>
#     all_docs.extend(docs)
    
# print(len(all_docs))
# print(all_docs[0].id_)
# print(all_docs[0].get_content())

In [5]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores.utils import filter_complex_metadata
import glob

pages = []
for file_path in glob.glob("data/*.pdf"):
    loader = PyPDFLoader(file_path)
    pages.extend(loader.load_and_split())
    
print(len(pages))

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=100)
chunks = text_splitter.split_documents(pages)
chunks = filter_complex_metadata(chunks)

print(len(chunks))

93
318


In [6]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings()
db = FAISS.from_documents(chunks, embedding)
db.save_local('faiss')

d:\anaconda3\envs\qwen\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [11]:
from langchain.llms import Ollama
from langchain.chains import RetrievalQA
from langchain import PromptTemplate

embedding = HuggingFaceEmbeddings()
    
model = Ollama(model='mistral')

query = input("\nQuery: ")

# Prompt
template = """Answer the following question from the pdf files,
{context}
Question: {question}
Helpful Answer:"""
    
QA_CHAIN_PROMPT = PromptTemplate(
        input_variables=["context", "question"],
        template=template,
    )
qa_chain = RetrievalQA.from_chain_type(
        model,
        retriever=db.as_retriever(),
        chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
        return_source_documents=True,
    )

result = qa_chain({"query": query})

print(result)

d:\anaconda3\envs\qwen\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


{'query': 'what is mixture of experts', 'result': ' The Mixture of Experts (MoE) refers to a type of neural network architecture where multiple "expert" networks work together to make predictions or decisions based on the input data. Each expert network specializes in solving different sub-problems, and a gating network determines which expert(s) should be used for a given input. This architecture allows for flexible and adaptive learning, as it can handle complex problems that may contain many sub-problems each requiring different experts. In this context, the MoE serves as a general-purpose neural network component.', 'source_documents': [Document(metadata={'source': 'data\\moe.pdf', 'page': 2}, page_content='the mixture-of-experts approach has been the subject of much research. Different types of expert\narchitectures hae been proposed such as SVMs (Collobert et al., 2002), Gaussian Processes (Tresp,\n2001; Theis & Bethge, 2015; Deisenroth & Ng, 2015), Dirichlet Processes (Shahbaba 

In [20]:
from langchain_core.prompts.chat import ChatPromptTemplate

template = """Answer the following question from the pdf files,
Question: {question}
Helpful Answer:"""

prompt = ChatPromptTemplate.from_template(template)

chain = prompt | model

print(chain.invoke({"question": "What is mixture of experts?"}))

 Mixture of Experts (MoE) is a hierarchical neural network model that is used to solve complex problems by delegating them to a set of specialized sub-networks called "experts." The idea is to have multiple models, each specialized in handling different parts of the input space.

In MoE, a gating network determines the weight of each expert based on the input and chooses the best or a combination of the experts to make predictions. This allows for more efficient and accurate modeling compared to having a single large model that attempts to handle all possible inputs directly. It is particularly useful in situations where data is diverse or complex, as it can adaptively allocate computational resources across the problem space.
